# Scratch pad StatsModels

The purpose of this notebook is to explore single-factor and multiple-factor regression using StatsModels.

In [ ]:
import pandas as pd
import matplotlib.cm as cm
import matplotlib.axes as axes
%matplotlib inline

In [ ]:
# Colour-blind colour definitions.
c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12 = cm.Paired.colors
# See "paired" in "qualitative colormaps"
# https://matplotlib.org/tutorials/colors/colormaps.html

In [ ]:
# Function to remove upper and right graphic borders.
def despine(ax: axes.Axes) -> None:
    'Remove the top and right spines of a graph'
    for spine in 'right', 'top':
        ax.spines[spine].set_color('none')